<a href="https://colab.research.google.com/github/pilipandr770/GOIT_diplom/blob/main/GOIT_Diplom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import json

# Завантаження даних з локального файлу
from google.colab import files
uploaded = files.upload()

# Визначення шляху до завантаженого файлу
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# Перегляд перших рядків даних
data.head()

# Видалення колонки ID, якщо така є
data = data.drop(columns=['ID', 'customerID'], errors='ignore')

# Обробка відсутніх значень
data = data.fillna(data.median())

# Перейменування колонки 'churn' для уніфікації
if 'churn' in data.columns:
    data.rename(columns={'churn': 'Churn'}, inplace=True)

# Кодування категоріальних змінних
data = pd.get_dummies(data, drop_first=True)

# Поділ на ознаки та цільову змінну
X = data.drop(columns=['Churn'])
y = data['Churn']

# Поділ на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створення та навчання моделей
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=5000, solver='lbfgs', penalty='l2'),
    'SVC': SVC(probability=True),
    'DecisionTree': DecisionTreeClassifier(random_state=42)
}

metrics = {}

for name, model in models.items():
    # Навчання моделі
    model.fit(X_train, y_train)

    # Прогнозування на тестовій вибірці
    y_pred = model.predict(X_test)

    # Оцінка моделі
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    metrics[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    # Виведення метрик моделі
    print(f'Model: {name}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print('------------------------')

    # Збереження моделі
    joblib.dump(model, f'/content/{name}_model.pkl')

# Збереження метрик оцінки моделей
with open('/content/model_metrics.json', 'w') as f:
    json.dump(metrics, f)

# Збереження назв ознак після підготовки X
feature_names = X.columns.tolist()
with open('/content/feature_names.json', 'w') as f:
    json.dump(feature_names, f)

# Збереження навчальних та тестових наборів даних
X_train.to_csv('/content/X_train.csv', index=False)
X_test.to_csv('/content/X_test.csv', index=False)
y_train.to_json('/content/y_train.json')
y_test.to_json('/content/y_test.json')

Saving archive (3) (1).zip to archive (3) (1).zip
Model: RandomForest
Accuracy: 0.9696990660671048
Precision: 0.9780937740199847
Recall: 0.9662110858010631
F1 Score: 0.9721161191749427
------------------------
Model: LogisticRegression
Accuracy: 0.8923555863023175
Precision: 0.9343005748699699
Recall: 0.8638319412806884
F1 Score: 0.8976854287217254
------------------------
Model: SVC
Accuracy: 0.7131096506399169
Precision: 0.7167263072838509
Recall: 0.7857504429258415
F1 Score: 0.749652882583761
------------------------
Model: DecisionTree
Accuracy: 0.9777931511587686
Precision: 0.9820679130103014
Recall: 0.9772209567198178
F1 Score: 0.9796384395813511
------------------------
